##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 추정기(Estimator)를 사용한 다중 작업자 훈련

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

> Warning: Estimators are not recommended for new code.  Estimators run `v1.Session`-style code which is more difficult to write correctly, and can behave unexpectedly, especially when combined with TF 2 code. Estimators do fall under [compatibility guarantees](https://tensorflow.org/guide/versions), but will receive no fixes other than security vulnerabilities. See the [migration guide](https://tensorflow.org/guide/migrate) for details.

## 개요

참고: `tf.distribute` API와 함께 추정기를 사용할 수는 있지만, `tf.distribute`와 함께 Keras를 사용하는 것을 추천합니다. [Keras를 사용한 다중 작업자 훈련](../../guide/multi_worker_with_keras.ipynb)을 참조하세요. `tf.distribute.Strategy`를 추정기와 사용하는 것은 부분적으로만 지원하고 있습니다.

이 튜토리얼은 `tf.estimator`를 이용한 분산 다중 작업자 훈련을 위해 `tf.distribute.Strategy`를 사용하는 방법을 보여줍니다. `tf.estimator`를 사용하여 코드를 작성하고 있고, 고성능의 장비 한 대로 다룰 수 있는 것보다 더 큰 작업을 수행하는 데 관심이 있다면 이 튜토리얼이 적합합니다.

시작하기 전에 [분산 전략](../../guide/distributed_training.ipynb) 가이드를 읽어주세요. 이 튜토리얼과 같은 모델을 사용하는 [다중 GPU 훈련 튜토리얼](./keras.ipynb)도 관련이 있습니다.


## 설정

먼저, 텐서플로를 설정하고 필요한 패키지들을 가져옵니다.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os, json

참고: TF2.4부터 다중 작업자 미러링 방법은 즉시 실행이 활성화된 경우(기본 설정) 추정기에서 오류를 일으킵니다. TF2.4의 오류는 `TypeError: cannot pickle '_thread.lock' object`입니다. 자세한 내용은 [이슈 #46556](https://github.com/tensorflow/tensorflow/issues/46556)을 참조하세요. 해결 방법은 즉시 실행을 비활성화하는 것입니다.

In [ ]:
tf.compat.v1.disable_eager_execution()

## 입력 함수

이 튜토리얼은 [TensorFlow 데이터세트](https://www.tensorflow.org/datasets)의 MNIST 데이터세트를 사용합니다. 코드 내용은 [다중 GPU 훈련 튜토리얼](./keras.ipynb)과 유사하지만 큰 차이점이 하나 있습니다. 바로 추정기를 사용하여 다중 작업자 훈련을 할 때는 데이터세트를 작업자 숫자대로 나누어 주어야 모델이 수렴합니다. 입력 데이터는 작업자 인덱스로 샤딩(shard)합니다. 그러면 각 작업자가 데이터세트의 `1/num_workers` 고유 부분을 처리합니다.

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

훈련을 수렴시키기 위한 또 다른 방법으로 각 작업자에서 데이터세트를 제각기 다른 시드 값으로 셔플하는 것도 있습니다.

## 다중 작업자 구성

[다중 GPU 훈련 튜토리얼](./keras.ipynb)과 비교할 때 가장 큰 차이 중 하나는 다중 워커를 설정하는 부분입니다. `TF_CONFIG` 환경 변수는 클러스터를 이루는 각 워커에 클러스터 설정을 지정하는 표준 방법입니다.

`TF_CONFIG`에는 `cluster`와 `task`라는 두 가지 구성 요소가 있습니다. `cluster`는 전체 클러스터, 다시 말해 클러스터에 속한 작업자와 매개변수 서버에 대한 정보를 제공합니다. `task`는 현재 작업에 대한 정보를 제공합니다. 첫 번째 구성 요소 `cluster`는 모든 작업자 및 매개변수 서버에 대해 동일하며 두 번째 구성 요소 `task`는 각 작업자 및 매개변수 서버에서 다르며 고유한 `type` 및 `index`를 지정합니다. 이 예제에서는 작업의 `type`이 `worker`이고, 작업의 `index`는 `0`입니다.

예를 들기 위해 이 튜토리얼에서는 두 개의 작업자를 `localhost`에 띄울 때의 `TF_CONFIG`를 보여드리겠습니다. 실제로는 외부 IP 주소 및 포트에 여러 작업자를 만들고 각 작업자에 대해 `TF_CONFIG`를 적절하게 설정합니다. 예를 들어 `index` 작업을 수정합니다.

경고: *Colab에서 다음 코드를 실행하지 마세요.* TensorFlow의 런타임은 지정된 IP 주소 및 포트에서 gRPC 서버를 생성하려고 시도하지만 실패할 가능성이 높습니다. 단일 시스템에서 여러 작업자를 테스트 실행하는 방법의 예는 이 튜토리얼의 [keras 버전](multi_worker_with_keras.ipynb)을 참조하세요.

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```


## 모델 정의하기

훈련을 위하여 레이어와 옵티마이저, 손실 함수를 정의하세요. 이 튜토리얼에서는 [다중 GPU 훈련 튜토리얼](./keras.ipynb)과 비슷하게 케라스 레이어로 모델을 정의합니다.

In [ ]:
LEARNING_RATE = 1e-4
def model_fn(features, labels, mode):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'logits': logits}
    return tf.estimator.EstimatorSpec(labels=labels, predictions=predictions)

  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
      learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(labels, logits)
  loss = tf.reduce_sum(loss) * (1. / BATCH_SIZE)
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss)

  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=optimizer.minimize(
          loss, tf.compat.v1.train.get_or_create_global_step()))

Note: 이 예제에서는 학습률이 고정되어있습니다. 하지만 실제로는 전역 배치 크기에 따라 학습률을 조정해야 할 수 있습니다.

## MultiWorkerMirroredStrategy

모델을 훈련하기 위하여 `tf.distribute.experimental.MultiWorkerMirroredStrategy`의 인스턴스를 사용하세요. `MultiWorkerMirroredStrategy`는 모든 워커의 각 장비에, 모델의 레이어에 있는 모든 변수의 복사본을 만듭니다. 이 전략은 `CollectiveOps`라는 수집을 위한 통신용 텐서플로 연산을 사용하여 그래디언트를 모으고, 변수들의 값을 동일하게 맞춥니다. [텐서플로로 분산 훈련하기](../../guide/distributed_training.ipynb)에 이 전략에 대한 더 자세한 내용이 있습니다.

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

## 모델 훈련 및 평가하기

다음으로, 추정기의 `RunConfig`에 분산 전략을 지정하십시오. 그리고 `tf.estimator.train_and_evaluate`로 훈련 및 평가를 합니다. 이 튜토리얼에서는 `train_distribute`로만 전략을 지정하였기 때문에 훈련 과정만 분산 처리합니다. `eval_distribute`를 지정하여 평가도 분산 처리할 수 있습니다.

In [ ]:
config = tf.estimator.RunConfig(train_distribute=strategy)

classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='/tmp/multiworker', config=config)
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

## 훈련 성능 최적화하기

이제 `tf.distribute.Strategy`에 의해 구동되는 모델과 다중 작업자 지원 Estimator가 있습니다. 다중 작업자 훈련의 성능을 최적화하기 위해 다음 방법을 시도할 수 있습니다.

- *배치 크기 늘리기:* 여기서 지정하는 배치 크기는 GPU당 크기입니다. 일반적으로, GPU 메모리 크기에 맞는 한 가장 크게 배치 크기를 잡는 것이 좋습니다.

- *변수 형변환:* 가능하면 변수를 `tf.float` 타입으로 바꾸세요.  공식 ResNet 모델의 [예제](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466)에서 어떻게 변환하는지 볼 수 있습니다.

- *집합 통신 구현을 사용하세요:* `MultiWorkerMirroredStrategy`는 여러 가지 [집합 통신 구현](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py)을 제공합니다.

    - `RING`은 장비 간 통신을 위하여 gRPC를 써서 링 네트워크 기반의 집합 통신을 구현한 것입니다.
    - `NCCL`은 [Nvidia의 NCCL](https://developer.nvidia.com/nccl)을 사용하여 수집 연산을 구현한 것입니다.
    - `AUTO`는 런타임이 알아서 고르도록 합니다.

    어떤 집합 구현이 가장 좋은지는 GPU의 숫자와 종류, 클러스터 장비 간 네트워크 연결 등에 따라 다를 수 있습니다. 런타임 자동 선택을 오버라이드하려면, `MultiWorkerMirroredStrategy` 생성자의 `communication` 인자에 적절한 값을 주면 됩니다. 예를 들어 `communication=tf.distribute.experimental.CollectiveCommunication.NCCL`과 같이 주면 됩니다.

가이드의 [성능 섹션](../../guide/function.ipynb)을 방문하여 TensorFlow 모델의 성능을 최적화하는 데 사용할 수 있는 다른 전략과 [도구](../../guide/profiler.md)에 대해 자세히 알아보세요.


## 다른 코드 예제

1. [처음부터 끝까지 살펴보는 예제](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy)에서는 tensorflow/ecosystem의 쿠버네티스(Kubernetes) 템플릿을 이용하여 다중 워커를 사용하여 훈련합니다. 이 예제에서는 케라스 모델을 만든 후, `tf.keras.estimator.model_to_estimator` API를 이용하여 추정기 모델로 변환합니다.
2. 많은 부분을 다중 분산 전략으로 실행할 수 있는 [공식 모델](https://github.com/tensorflow/models/blob/master/official/resnet/imagenet_main.py).
